In [63]:
from datasets import list_datasets, load_dataset
import pandas as pd
dataset = load_dataset('winograd_wsc', 'wsc285')
df = pd.DataFrame(dataset['test'])
df['answer'] = df.apply(lambda x: x['options'][x['label']], axis=1)

df_3 = pd.read_csv("gpt3_joined.csv")
df_3.columns = ['gpt3']

import re
df_4 = pd.read_csv("gpt4_answers", header=None)
df_4.columns = ['gpt4']
df_4['gpt4'] = df_4['gpt4'].apply(lambda x: re.sub("'", '', x))

df['gpt4'] = df_4['gpt4']
df['gpt3'] = df_3['gpt3']


from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def evaluate(df, pred_col):
    df[pred_col + "_similarity"] = df.apply(lambda x: similar(x['answer'].lower(), x[pred_col].lower()), axis=1)
    df[pred_col + '_correct'] = 0
    df.loc[df[pred_col + "_similarity"] > 0.75, pred_col + '_correct'] = 1
    print(df[pred_col + '_correct'].mean())
    return df

df = evaluate(df, 'gpt4')
df = evaluate(df, 'gpt3')

Found cached dataset winograd_wsc (/Users/denis/.cache/huggingface/datasets/winograd_wsc/wsc285/0.0.0/0651311f3b6dda14889d9a063030a02458395ee50ab9f41cca4cd5a89c0c3dce)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 66.14it/s]


0.9438596491228071
0.6877192982456141


In [71]:
pd.set_option('max_colwidth', 800)
display(df.loc[df['gpt4_correct'] == 0][['text', 'quote', 'options', 'answer']])

,text,quote,options,answer
50,"Joe's uncle can still beat him at tennis, even though he is 30 years younger.",he is 30 years younger.,"[Joe, Joe's uncle]",Joe
51,"Joe's uncle can still beat him at tennis, even though he is 30 years older.",he is 30 years older.,"[Joe, Joe's uncle]",Joe's uncle
192,"I stuck a pin through a carrot. When I pulled the pin out, it left a hole.",it left a hole,"[The pin, The carrot]",The pin
193,"I stuck a pin through a carrot. When I pulled the pin out, it had a hole.",it had a hole,"[The pin, The carrot]",The carrot
206,Sam Goodman's biography of the Spartan general Xenophanes conveys a vivid sense of the difficulties he faced in his research.,difficulties he faced,"[Goodman, Xenophanes]",Goodman
217,Jim signaled the barman and gestured toward his bathroom key.,his bathroom key,"[Jim, The barman]",The barman
218,"Dan took the rear seat while Bill claimed the front because his ""Dibs!"" was slow.","his ""Dibs!""","[Dan, Bill]",Dan
219,"Dan took the rear seat while Bill claimed the front because his ""Dibs!"" was quicker.","his ""Dibs!""","[Dan, Bill]",Bill
233,"Patting her back, the woman smiled at the girl.",her back,"[The woman, The girl]",The girl
247,"Pam's parents came home and found her having sex with her boyfriend, Paul. They were embarrassed about it.",They were embarrassed,"[Pam's parents, Pam and Paul]",Pam and Paul
